This notebook is meant to accompany the paper **A characteristics-based approximation for wave scattering from an arbitrary obstacle in one dimension**  by Jithin D. George, David I. Ketcheson, and Randall J. LeVeque.  The notebook includes some details of useful calculations as well as code to reproduce examples in the paper.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from clawpack import pyclaw, riemann
from clawpack.visclaw import ianimate
from scipy import integrate
from numba import jit
import matplotlib
matplotlib.rcParams.update({'font.size': 16})

# Reflection and transmission of a square wave
The code below is specialized to the case of a square wave initial condition:
$$p(x,t=0) = \begin{cases} 1 & -w \le x \le 0 \\ 0 & \text{elsewhere} \end{cases}$$
As in the paper, the initial velocity is such that this square wave is purely right-going.
The square wave is discussed extensively in the paper and is a convenient choice since it
allows us to perform one integration analytically, reducing by one the dimension of the quadrature
required.  In the code below, we've also specialized (for simplicity) to the case where $c(x)$ is
a linear function; $Z(x)$ can be more complicated.  There are some commented-out snippets that will
work for arbitrary $c(x)$.

In [ ]:
@jit
def c_fun(x):
    "Sound speed c(x).  If this function is changed, then X(t) and T(x) must be modified as well."
    return (x<x_0)*c_l + (x>x_r)*c_r + (x>=x_0)*(x<=x_r)*(c_l + s*(x-x_0))

@jit
def Z_fun(x):
    "Impedance Z(x).  If this is changed, Z_prime(x) must also be modified."
    return (x<x_0)*Z_l + (x>x_r)*Z_r + (x>=x_0)*(x<=x_r)*(Z_l + sZ*(x-x_0)) \
            + oscillate*(x>x_0)*(x<x_r)*np.sin(10*np.pi*x)

@jit
def Z_prime(x):
    "Derivative of Z(x)."
    if oscillate==0:
        return sZ
    else:
        return sZ + oscillate*10*np.pi*np.cos(10*np.pi*x)

def X_rhs(X,t):
    "ODE for X(t) (unreflected characteristic path)."
    return c_fun(X)


# for arbitrary c(x):
#tvals = np.linspace(0,t_r)
#Xvals = integrate.odeint(X_rhs, x_0, tvals).squeeze()

@jit
def X(t):
    "x-location of a characteristic that leaves x=0 at t=0 (going to the right) and is not reflected."
    # This version works for any c(x):
    #return (t<=0)*0 + (t>=t_r)*x_r + (t>0)*(t<t_r)*pwlinear(tvals, Xvals, Xvals, t)
    # Only for linearly-varying sound speed:
    return (t<=0)*0 + (t>=t_r)*x_r + (t>0)*(t<t_r)*c_l/s*(np.exp(s*t)-1)

@jit
def T(x):
    "Time taken for a characteristic to reach x (from zero)."
    # This version works for any c(x):
    #return pwlinear(Xvals, tvals, tvals, x)
    # Only for linear c(x):
    return 1./s*np.log(s*x/c_l+1)

# Twice-reflected waves: $T_2$

The integral for twice-reflected waves
takes the form
\begin{align*}
    T_2 & = -C_G \int_0^{x_+} \int_0^{x_1} \eta_0(\xi_T({\mathbf x},t)) r(x_1) r(x_2) dx_2 dx_1 \\
        & = -C_G \frac{1}{4} \int_0^{x_r} \frac{Z'(x_1)}{Z(x_1)} \int_{\min(x_1,\alpha_2(t,x_1))}^{\min(x_1,\beta_2(t,x_1))}  \frac{Z'(x_2)}{Z(x_2)} dx_2 dx_1 \\
        & = -C_G \int_0^{x_r} \frac{1}{4} \frac{Z'(x_1)}{Z(x_1)} \log\left(\frac{Z(\min(x_1,\beta_2))}{Z(\min(x_1,\alpha_2))}\right)dx_1.
\end{align*}
The bounds of integration turn out to be
\begin{align*}
    \alpha(t,x_1) & = X\left(\tau(x_1) - \frac{t-t_+}{2}\right) \\
    \beta(t,x_1)  & = X\left(\tau(x_1) - \frac{t-t_+-w/c_-}{2}\right)
\end{align*}


In [ ]:
def make_trans_fcn(c_l, c_r, Z_l, Z_r, x_0, x_r, oscillate):
    def transmitted_wave(x,t,w=1.,N=2):

        t_w = w/c_l

        def T0(t,w):
            "First transmitted characteristics (no reflections)."
            x_init = (t-t_r)*(-c_l)
            return (x_init<x_0)*(x_init>-w)*np.sqrt(Z_r/Z_l)

        # T_2: Twice-reflected waves (eventually transmitted)
        def T2(t):
            "Twice-reflected waves"

            def alpha2(t):
                "Lower limit of integration in x1."
                return (t<(3*t_r+t_w))*X((t-t_r-t_w)/2.)

            def gamma2(x1):  # Lower limit of integration for x2
                T1 = T(x1)
                return (t<t_r+2*T1)*(X(T1-(t-t_r)/2.)) + 0.*t

            def integrand(x1):
                x2_lower = X(T(x1)-(t-t_r)/2)
                x2_upper = X(T(x1)-(t-t_r-t_w)/2)
                # Faster, but works only for linear Z(x):
                #return -sZ/(4.*Z_fun(x1))*(np.log(Z_fun(min(x2_upper,x1)))-np.log(Z_fun(min(x2_lower,x1))))
                # Slower, but works for any Z(x):
                return -Z_prime(x1)/(4.*Z_fun(x1))*(np.log(Z_fun(min(x2_upper,x1)))-np.log(Z_fun(min(x2_lower,x1))))

            if (t<t_r) or (t>3*t_r+t_w):
                return 0
            else:
                return (Z_r/Z_l)**(1./2)*integrate.quad(integrand,0,x_r,epsabs=1.e-3,epsrel=1.e-3)[0]

        def T4(t):
            "Four-times reflected waves"

            def integrand4(x_3, x_2, x_1):
                x4_lower = X(T(x_1)+T(x_3)-T(x_2)-(t-t_r)/2)
                x4_upper = X(T(x_1)+T(x_3)-T(x_2)-(t-t_r-t_w)/2)
                # Faster, but works only for linear Z(x):
                #return -sZ**3/16*np.log(Z_fun(min(x4_lower,x_3))/Z_fun(min(x4_upper,x_3)))/(Z_fun(x_1)*Z_fun(x_2)*Z_fun(x_3))
                # Slower, but works for any Z(x):
                return -Z_prime(x_1)*Z_prime(x_2)*Z_prime(x_3)/16*np.log(Z_fun(min(x4_lower,x_3))/Z_fun(min(x4_upper,x_3)))/(Z_fun(x_1)*Z_fun(x_2)*Z_fun(x_3))

            def zerofun(x):
                return 0

            def identity(x_1):
                return x_1

            def identity2(x_1, x_2):
                return x_2

            def x_r_fun(x_1, x_2):
                return x_r

            if (t<t_r) or (t>5*t_r+t_w):
                return 0
            else:
                return (Z_r/Z_l)**(1/2)*integrate.tplquad(integrand4,0,x_r,zerofun,identity,identity2,x_r_fun,
                                                          epsabs=1.e-3,epsrel=1.e-3)[0]

        if N>4: raise Exception('Transmitted terms beyond N=4 have not been coded.')
        T_0 = T0(t-(x-x_r)/c_r,w)
        T_2 = np.zeros_like(x)
        T_4 = np.zeros_like(x)
        if N>=2:
            for i, xx in enumerate(x):
                tt = t-(xx-x_r)/c_r
                if tt>t_r and tt<(3*t_r+t_w):
                    T_2[i] = T2(tt)
                if tt>t_r and tt<(5*t_r+t_w) and N>=4:
                    T_4[i] = T4(tt)

        return (x>x_r)*(T_0 + T_2 + T_4)
    
    return transmitted_wave

# Once-reflected waves: $R_1$
The integral for once-reflected waves
takes the form
\begin{align*}
    R_1 & = \int_0^{x_+} \eta_0(\xi_R({\mathbf x},t)) r(x_1) dx_1 \\
        & = \frac{1}{2} \int_{\alpha_1(t)}^{X(t/2)} \frac{Z'(x_1)}{Z(x_1)} dx_1 \\
        & = \frac{1}{2} \log\left(\frac{Z(X(t/2))}{Z(\alpha(t))}\right).
\end{align*}
The bounds of integration are determined by considering for what values of $x_1$
the initial value $\eta_0(\xi_R(x_1,t))$ is nonzero.  The upper bound, $X(t/2)$, is
simply the furthest point a reflected characteristic path could have reached
in order to return to $x=0$ at time $t$.  It turns out that the lower bound is
\begin{align*}
    \alpha(t) = X((t-w/c_-)/2).
\end{align*}


# Thrice-reflected waves: $R_3$

Similarly, for the thrice-reflected waves we find
\begin{align*}
    R_3(t) & = -\int_0^{x_+} \int_0^{x_1} \int_{x_2}^{x_+} \eta_0(\xi_R({\mathbf x},t)) r(x_1) r(x_2) r(x_3) dx_3 dx_2 dx_1 \\
        & = -\frac{1}{8}\int_0^{X(t/2)} \int_0^{x_1} \int_{\max(x_2,\alpha_3)}^{\max(x_2,\beta_3)} \frac{Z'(x_1)}{Z(x_1)} \frac{Z'(x_2)}{Z(x_2)} \frac{Z'(x_3)}{Z(x_3)} dx_3 dx_2 dx_1 \\
        & = -\frac{1}{8}\int_0^{X(t/2)} \int_0^{x_1} \frac{Z'(x_1)}{Z(x_1)} \frac{Z'(x_2)}{Z(x_2)} \log\left(\frac{Z(\max(x_2, \beta_3))}{Z(\max(x_2, \alpha_3))}\right) dx_2 dx_1.
\end{align*}
Here the bounds of integration $\alpha_3, \beta_3$ give the range of permissible reflection
points $x_3$, such that a characteristic path originating from $[-w,0]$ reaches $x=0$ after
three reflections.  These turn out to be
\begin{align*}
    \alpha(t,x_1,x_2) & = X\left(\frac{t-w/c_+}{2}+\tau(x_2) - \tau(x_1)\right) \\
    \beta(t,x_1,x_2)  & = X\left(\frac{t}{2}+\tau(x_2) - \tau(x_1)\right).
\end{align*}


In [ ]:
def make_refl_fcn(c_l, c_r, Z_l, Z_r, x_0, x_r, oscillate):
    def reflected_wave(x,t,w=1.,N=3):
        # Lower and upper limits of integration for first reflections integral
        t_w = w/c_l

        def alpha(t):
            return (t < w/c_l)*0 + (t < (2*t_r + t_w))*X((t-t_w)/2.)

        def beta(t):
            return (t < 2*t_r)*X(t/2.) + (t > 2*t_r)*(t < (2*t_r + t_w))*x_r

        def R_1(t):
            "Amplitude of first reflections at x=0 as a function of time."
            return (t>0) * 0.5 * np.log(Z_fun(beta(t))/Z_fun(alpha(t)))


        # R_3: Thrice-reflected waves (eventually reflected)
        def R3(t):
            "Thrice-reflected waves"
            t_w = w/c_l

            def integrand(x2,x1):
                x3_upper = X(t/2-T(x1)+T(x2))
                x3_lower = X((t-t_w)/2 + T(x2) - T(x1))
                #return sZ**2/8.*np.log(Z_fun(max(x2,x3_lower))/Z_fun(max(x2,x3_upper)))/(Z_fun(x1)*Z_fun(x2))
                return Z_prime(x1)*Z_prime(x2)/8.*np.log(Z_fun(max(x2,x3_lower))/Z_fun(max(x2,x3_upper)))/(Z_fun(x1)*Z_fun(x2))

            def zerofun(x1):
                return 0

            def identity(x1):
                return x1

            if (t>(4*t_r + t_w)):
                return 0
            else:
                b = X(t/2)
                return integrate.dblquad(integrand,0,b,zerofun,identity,epsabs=1.e-3,epsrel=1.e-3)[0]

        if N>4: raise Exception('Reflection terms beyond n=3 have not been coded.')
        R_3 = np.zeros_like(x)
        if N>=3:
            for i, xx in enumerate(x):
                tt = t+xx/c_l
                R_3[i] = R3(tt)        

        return (x<x_0)*(R_1(t+x/c_l)+R_3)
    
    return reflected_wave

# Code to solve the problem with PyClaw
We'll use this as a reference for comparison.

In [ ]:
def setup(mx=1000, w=1., Z_l = 1., Z_r = 2., c_l=1., c_r=1., x_0 = 0., x_r = 1., oscillate=False):
    from clawpack.riemann.acoustics_variable_1D_constants import impedance, sound_speed, velocity, pressure
    
    riemann_solver = riemann.acoustics_variable_1D
    solver = pyclaw.ClawSolver1D(riemann_solver)

    # Boundary conditions
    solver.bc_lower[0] = pyclaw.BC.extrap
    solver.bc_upper[0] = pyclaw.BC.extrap
    solver.aux_bc_lower[0] = pyclaw.BC.extrap
    solver.aux_bc_upper[0] = pyclaw.BC.extrap

    xlower=x_0-x_r*6; xupper=x_0+x_r*3
    x = pyclaw.Dimension(xlower,xupper,mx)
    domain = pyclaw.Domain(x)
    state = pyclaw.State(domain,solver.num_eqn,2)
    xc=state.grid.x.centers

    #Initialize q and aux
    if x_r - x_0 == 0:
        Z = Z_l*(xc<x_0) + Z_r*(xc>=x_r)
        c = c_l*(xc<x_0) + c_r*(xc>=x_r)
    else:
        Z = Z_l*(xc<x_0) + Z_r*(xc>x_r) + (Z_l + (Z_r-Z_l)*(xc-x_0)/(x_r-x_0))*(xc>=x_0)*(xc<=x_r)
        c = c_l*(xc<x_0) + c_r*(xc>x_r) + (c_l + (c_r-c_l)*(xc-x_0)/(x_r-x_0))*(xc>=x_0)*(xc<=x_r)
        
    if oscillate:
        Z = Z + oscillate*(xc>x_0)*(xc<x_r)*np.sin(10*np.pi*xc)
        
    state.aux[impedance,:] = Z # Impedance
    state.aux[sound_speed,:] = c # Sound speed

    state.q[pressure,:] = (xc>x_0-w)*(xc<x_0)  # p
    state.q[velocity,:] = state.q[0,:]/Z        # u

    claw = pyclaw.Controller()
    claw.solution = pyclaw.Solution(state,domain)

    claw.output_style = 1
    claw.num_output_times = 10
    claw.tfinal =  3.*t_r
    claw.solver = solver
    claw.keep_copy = True
    claw.output_format = None
    
    return claw, Z, c

# Figure 8(a)

In [ ]:
c_l = 2.
c_r = 1.
Z_l = 0.5
Z_r = 1.
x_0 = 0.
x_r = 1.  # Slope extends from x_0 to x_r
s = (c_r-c_l)/(x_r-x_0)  # slope c
sZ = (Z_r-Z_l)/(x_r-x_0)
t_r = 1./s*np.log(s*x_r/c_l+1)  # crossing time
oscillate = 0.

transmitted_wave = make_trans_fcn(c_l, c_r, Z_l, Z_r, x_0, x_r, oscillate)
reflected_wave = make_refl_fcn(c_l, c_r, Z_l, Z_r, x_0, x_r, oscillate)

In [ ]:
w=100.
claw, Z, c = setup(mx=10000,w=w,Z_l=Z_l,Z_r=Z_r,
                c_l=c_l,c_r=c_r,x_0=x_0,x_r=x_r,oscillate=oscillate)
xc = claw.solution.state.grid.x.centers

claw.run()

In [ ]:
ianimate.ianimate(claw)

In [ ]:
N = 2
i = -1
skip = 100
q = claw.frames[i].q
t = claw.frames[i].t
x_coarse = xc[::skip]
wave2 = reflected_wave(x_coarse,t,w,N=N) + transmitted_wave(x_coarse,t,w,N=N)
wave2 = wave2 + 1*(x_coarse<x_0)

wave4 = reflected_wave(x_coarse,t,w,N=4) + transmitted_wave(x_coarse,t,w,N=4)
wave4 = wave4 + 1*(x_coarse<x_0)

In [ ]:
plt.figure(figsize=(10,6))
leftpart = np.where(x_coarse<0)
rightpart = np.where(x_coarse>x_r)

plt.plot(xc,q[0,:],'-k',zorder=11);
plt.plot(x_coarse[leftpart],wave2[leftpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[leftpart],wave4[leftpart],'b',lw=3,alpha=0.5);
plt.plot(x_coarse[rightpart],wave2[rightpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[rightpart],wave4[rightpart],'b',lw=3,alpha=0.5);
ax = plt.gca()
ymax, ymin = ax.get_ylim()
xmid = np.linspace(0,1)
plt.fill_between(xmid,ymax,ymin,color='lightgray',alpha=1.,zorder=10)
plt.autoscale(enable=True, axis='y', tight=True)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend(['exact','Up to 2 reflections','Up to 4 reflections'],fontsize=15,loc=3);

# Figure 8(b)

In [ ]:
w=1.
claw, Z, c = setup(mx=10000,w=w,Z_l=Z_l,Z_r=Z_r,
                c_l=c_l,c_r=c_r,x_0=x_0,x_r=x_r,oscillate=oscillate)
xc = claw.solution.state.grid.x.centers

claw.run()

In [ ]:
N = 2
i = -1
skip = 100
q = claw.frames[i].q
t = claw.frames[i].t
x_coarse = xc[::skip]
wave2 = reflected_wave(x_coarse,t,w,N=N) + transmitted_wave(x_coarse,t,w,N=N)

wave4 = reflected_wave(x_coarse,t,w,N=4) + transmitted_wave(x_coarse,t,w,N=4)

In [ ]:
plt.figure(figsize=(10,6))
leftpart = np.where(x_coarse<0)
rightpart = np.where(x_coarse>x_r)

plt.plot(xc,q[0,:],'-k',zorder=11);
plt.plot(x_coarse[leftpart],wave2[leftpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[leftpart],wave4[leftpart],'b',lw=3,alpha=0.5);
plt.plot(x_coarse[rightpart],wave2[rightpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[rightpart],wave4[rightpart],'b',lw=3,alpha=0.5);
ax = plt.gca()
ymax, ymin = ax.get_ylim()
xmid = np.linspace(0,1)
plt.fill_between(xmid,ymax,ymin,color='lightgray',alpha=1.,zorder=10)
plt.autoscale(enable=True, axis='y', tight=True)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend(['exact','Up to 2 reflections','Up to 4 reflections'],fontsize=15,loc='best');

# Figure 9(a)

In [ ]:
Z_l = 1./8
sZ = (Z_r-Z_l)/(x_r-x_0)

In [ ]:
@jit
def c_fun(x):
    "Sound speed c(x).  If this function is changed, then X(t) and T(x) must be modified as well."
    return (x<x_0)*c_l + (x>x_r)*c_r + (x>=x_0)*(x<=x_r)*(c_l + s*(x-x_0))

@jit
def Z_fun(x):
    "Impedance Z(x).  If this is changed, Z_prime(x) must also be modified."
    return (x<x_0)*Z_l + (x>x_r)*Z_r + (x>=x_0)*(x<=x_r)*(Z_l + sZ*(x-x_0)) \
            + oscillate*(x>x_0)*(x<x_r)*np.sin(10*np.pi*x)

@jit
def Z_prime(x):
    "Derivative of Z(x)."
    if oscillate==0:
        return sZ
    else:
        return sZ + oscillate*10*np.pi*np.cos(10*np.pi*x)

def X_rhs(X,t):
    "ODE for X(t) (unreflected characteristic path)."
    return c_fun(X)


# for arbitrary c(x):
#tvals = np.linspace(0,t_r)
#Xvals = integrate.odeint(X_rhs, x_0, tvals).squeeze()

@jit
def X(t):
    "x-location of a characteristic that leaves x=0 at t=0 (going to the right) and is not reflected."
    # This version works for any c(x):
    #return (t<=0)*0 + (t>=t_r)*x_r + (t>0)*(t<t_r)*pwlinear(tvals, Xvals, Xvals, t)
    # Only for linearly-varying sound speed:
    return (t<=0)*0 + (t>=t_r)*x_r + (t>0)*(t<t_r)*c_l/s*(np.exp(s*t)-1)

@jit
def T(x):
    "Time taken for a characteristic to reach x (from zero)."
    # This version works for any c(x):
    #return pwlinear(Xvals, tvals, tvals, x)
    # Only for linear c(x):
    return 1./s*np.log(s*x/c_l+1)

In [ ]:
transmitted_wave = make_trans_fcn(c_l, c_r, Z_l, Z_r, x_0, x_r, oscillate)
reflected_wave = make_refl_fcn(c_l, c_r, Z_l, Z_r, x_0, x_r, oscillate)

In [ ]:
w=100.
claw, Z, c = setup(mx=10000,w=w,Z_l=Z_l,Z_r=Z_r,
                c_l=c_l,c_r=c_r,x_0=x_0,x_r=x_r,oscillate=oscillate)
xc = claw.solution.state.grid.x.centers

claw.run()

In [ ]:
N = 2
i = -1
skip = 100
q = claw.frames[i].q
t = claw.frames[i].t
x_coarse = xc[::skip]
wave2 = reflected_wave(x_coarse,t,w,N=N) + transmitted_wave(x_coarse,t,w,N=N)
wave2 = wave2 + 1*(x_coarse<x_0)

wave4 = reflected_wave(x_coarse,t,w,N=4) + transmitted_wave(x_coarse,t,w,N=4)
wave4 = wave4 + 1*(x_coarse<x_0)

In [ ]:
plt.figure(figsize=(10,6))
leftpart = np.where(x_coarse<0)
rightpart = np.where(x_coarse>x_r)

plt.plot(xc,q[0,:],'-k',zorder=11);
plt.plot(x_coarse[leftpart],wave2[leftpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[leftpart],wave4[leftpart],'b',lw=3,alpha=0.5);
plt.plot(x_coarse[rightpart],wave2[rightpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[rightpart],wave4[rightpart],'b',lw=3,alpha=0.5);
ax = plt.gca()
ymax, ymin = ax.get_ylim()
xmid = np.linspace(0,1)
plt.fill_between(xmid,ymax,ymin,color='lightgray',alpha=1.,zorder=10)
plt.autoscale(enable=True, axis='y', tight=True)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend(['exact','Up to 2 reflections','Up to 4 reflections'],fontsize=15,loc='best');

# Figure 9(b)

In [ ]:
w=1.
claw, Z, c = setup(mx=10000,w=w,Z_l=Z_l,Z_r=Z_r,
                c_l=c_l,c_r=c_r,x_0=x_0,x_r=x_r,oscillate=oscillate)
xc = claw.solution.state.grid.x.centers

claw.run()

In [ ]:
N = 2
i = -1
skip = 100
q = claw.frames[i].q
t = claw.frames[i].t
x_coarse = xc[::skip]
wave2 = reflected_wave(x_coarse,t,w,N=N) + transmitted_wave(x_coarse,t,w,N=N)

wave4 = reflected_wave(x_coarse,t,w,N=4) + transmitted_wave(x_coarse,t,w,N=4)

In [ ]:
plt.figure(figsize=(10,6))
leftpart = np.where(x_coarse<0)
rightpart = np.where(x_coarse>x_r)

plt.plot(xc,q[0,:],'-k',zorder=11);
plt.plot(x_coarse[leftpart],wave2[leftpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[leftpart],wave4[leftpart],'b',lw=3,alpha=0.5);
plt.plot(x_coarse[rightpart],wave2[rightpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[rightpart],wave4[rightpart],'b',lw=3,alpha=0.5);
ax = plt.gca()
ymax, ymin = ax.get_ylim()
xmid = np.linspace(0,1)
plt.fill_between(xmid,ymax,ymin,color='lightgray',alpha=1.,zorder=10)
plt.autoscale(enable=True, axis='y', tight=True)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend(['exact','Up to 2 reflections','Up to 4 reflections'],fontsize=15,loc='best');

# Figure 10(a)

In [ ]:
Z_l = 1.
Z_r = 20.
sZ = (Z_r-Z_l)/(x_r-x_0)

In [ ]:
@jit
def c_fun(x):
    "Sound speed c(x).  If this function is changed, then X(t) and T(x) must be modified as well."
    return (x<x_0)*c_l + (x>x_r)*c_r + (x>=x_0)*(x<=x_r)*(c_l + s*(x-x_0))

@jit
def Z_fun(x):
    "Impedance Z(x).  If this is changed, Z_prime(x) must also be modified."
    return (x<x_0)*Z_l + (x>x_r)*Z_r + (x>=x_0)*(x<=x_r)*(Z_l + sZ*(x-x_0)) \
            + oscillate*(x>x_0)*(x<x_r)*np.sin(10*np.pi*x)

@jit
def Z_prime(x):
    "Derivative of Z(x)."
    if oscillate==0:
        return sZ
    else:
        return sZ + oscillate*10*np.pi*np.cos(10*np.pi*x)

def X_rhs(X,t):
    "ODE for X(t) (unreflected characteristic path)."
    return c_fun(X)


# for arbitrary c(x):
#tvals = np.linspace(0,t_r)
#Xvals = integrate.odeint(X_rhs, x_0, tvals).squeeze()

@jit
def X(t):
    "x-location of a characteristic that leaves x=0 at t=0 (going to the right) and is not reflected."
    # This version works for any c(x):
    #return (t<=0)*0 + (t>=t_r)*x_r + (t>0)*(t<t_r)*pwlinear(tvals, Xvals, Xvals, t)
    # Only for linearly-varying sound speed:
    return (t<=0)*0 + (t>=t_r)*x_r + (t>0)*(t<t_r)*c_l/s*(np.exp(s*t)-1)

@jit
def T(x):
    "Time taken for a characteristic to reach x (from zero)."
    # This version works for any c(x):
    #return pwlinear(Xvals, tvals, tvals, x)
    # Only for linear c(x):
    return 1./s*np.log(s*x/c_l+1)

In [ ]:
transmitted_wave = make_trans_fcn(c_l, c_r, Z_l, Z_r, x_0, x_r, oscillate)
reflected_wave = make_refl_fcn(c_l, c_r, Z_l, Z_r, x_0, x_r, oscillate)

In [ ]:
w=100.
claw, Z, c = setup(mx=10000,w=w,Z_l=Z_l,Z_r=Z_r,
                c_l=c_l,c_r=c_r,x_0=x_0,x_r=x_r,oscillate=oscillate)
xc = claw.solution.state.grid.x.centers

claw.run()

In [ ]:
N = 2
i = -1
skip = 100
q = claw.frames[i].q
t = claw.frames[i].t
x_coarse = xc[::skip]
wave2 = reflected_wave(x_coarse,t,w,N=N) + transmitted_wave(x_coarse,t,w,N=N)
wave2 = wave2 + 1*(x_coarse<x_0)

wave4 = reflected_wave(x_coarse,t,w,N=4) + transmitted_wave(x_coarse,t,w,N=4)
wave4 = wave4 + 1*(x_coarse<x_0)

In [ ]:
plt.figure(figsize=(10,6))
leftpart = np.where(x_coarse<0)
rightpart = np.where(x_coarse>x_r)

plt.plot(xc,q[0,:],'-k',zorder=11);
plt.plot(x_coarse[leftpart],wave2[leftpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[leftpart],wave4[leftpart],'b',lw=3,alpha=0.5);
plt.plot(x_coarse[rightpart],wave2[rightpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[rightpart],wave4[rightpart],'b',lw=3,alpha=0.5);
ax = plt.gca()
ymax, ymin = ax.get_ylim()
xmid = np.linspace(0,1)
plt.fill_between(xmid,ymax,ymin,color='lightgray',alpha=1.,zorder=10)
plt.autoscale(enable=True, axis='y', tight=True)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend(['exact','Up to 2 reflections','Up to 4 reflections'],fontsize=15,loc='best');

# Figure 10(b)

In [ ]:
w=1.
claw, Z, c = setup(mx=10000,w=w,Z_l=Z_l,Z_r=Z_r,
                c_l=c_l,c_r=c_r,x_0=x_0,x_r=x_r,oscillate=oscillate)
xc = claw.solution.state.grid.x.centers

claw.run()

In [ ]:
N = 2
i = -1
skip = 100
q = claw.frames[i].q
t = claw.frames[i].t
x_coarse = xc[::skip]
wave2 = reflected_wave(x_coarse,t,w,N=N) + transmitted_wave(x_coarse,t,w,N=N)

wave4 = reflected_wave(x_coarse,t,w,N=4) + transmitted_wave(x_coarse,t,w,N=4)

In [ ]:
plt.figure(figsize=(10,6))
leftpart = np.where(x_coarse<0)
rightpart = np.where(x_coarse>x_r)

plt.plot(xc,q[0,:],'-k',zorder=11);
plt.plot(x_coarse[leftpart],wave2[leftpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[leftpart],wave4[leftpart],'b',lw=3,alpha=0.5);
plt.plot(x_coarse[rightpart],wave2[rightpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[rightpart],wave4[rightpart],'b',lw=3,alpha=0.5);
ax = plt.gca()
ymax, ymin = ax.get_ylim()
xmid = np.linspace(0,1)
plt.fill_between(xmid,ymax,ymin,color='lightgray',alpha=1.,zorder=10)
plt.autoscale(enable=True, axis='y', tight=True)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend(['exact','Up to 2 reflections','Up to 4 reflections'],fontsize=15,loc='best');

# Figure 11(a)
These last two take quite a long time to run.

In [ ]:
c_l = 2.
c_r = 1.
Z_l = 1./4
Z_r = 1.
x_0 = 0.
x_r = 1.  # Slope extends from x_0 to x_r
s = (c_r-c_l)/(x_r-x_0)  # slope c
sZ = (Z_r-Z_l)/(x_r-x_0)
t_r = 1./s*np.log(s*x_r/c_l+1)  # crossing time
oscillate = 0.1

In [ ]:
@jit
def c_fun(x):
    "Sound speed c(x).  If this function is changed, then X(t) and T(x) must be modified as well."
    return (x<x_0)*c_l + (x>x_r)*c_r + (x>=x_0)*(x<=x_r)*(c_l + s*(x-x_0))

@jit
def Z_fun(x):
    "Impedance Z(x).  If this is changed, Z_prime(x) must also be modified."
    return (x<x_0)*Z_l + (x>x_r)*Z_r + (x>=x_0)*(x<=x_r)*(Z_l + sZ*(x-x_0)) \
            + oscillate*(x>x_0)*(x<x_r)*np.sin(10*np.pi*x)

@jit
def Z_prime(x):
    "Derivative of Z(x)."
    if oscillate==0:
        return sZ
    else:
        return sZ + oscillate*10*np.pi*np.cos(10*np.pi*x)

def X_rhs(X,t):
    "ODE for X(t) (unreflected characteristic path)."
    return c_fun(X)


# for arbitrary c(x):
#tvals = np.linspace(0,t_r)
#Xvals = integrate.odeint(X_rhs, x_0, tvals).squeeze()

@jit
def X(t):
    "x-location of a characteristic that leaves x=0 at t=0 (going to the right) and is not reflected."
    # This version works for any c(x):
    #return (t<=0)*0 + (t>=t_r)*x_r + (t>0)*(t<t_r)*pwlinear(tvals, Xvals, Xvals, t)
    # Only for linearly-varying sound speed:
    return (t<=0)*0 + (t>=t_r)*x_r + (t>0)*(t<t_r)*c_l/s*(np.exp(s*t)-1)

@jit
def T(x):
    "Time taken for a characteristic to reach x (from zero)."
    # This version works for any c(x):
    #return pwlinear(Xvals, tvals, tvals, x)
    # Only for linear c(x):
    return 1./s*np.log(s*x/c_l+1)

In [ ]:
transmitted_wave = make_trans_fcn(c_l, c_r, Z_l, Z_r, x_0, x_r, oscillate)
reflected_wave = make_refl_fcn(c_l, c_r, Z_l, Z_r, x_0, x_r, oscillate)

In [ ]:
w=100.
claw, Z, c = setup(mx=40000,w=w,Z_l=Z_l,Z_r=Z_r,
                c_l=c_l,c_r=c_r,x_0=x_0,x_r=x_r,oscillate=oscillate)
xc = claw.solution.state.grid.x.centers

claw.run()

In [ ]:
ianimate.ianimate(claw)

In [ ]:
i = -1
skip = 100
q = claw.frames[i].q
t = claw.frames[i].t
x_coarse = xc[::skip]
wave2 = reflected_wave(x_coarse,t,w,N=2) + transmitted_wave(x_coarse,t,w,N=2)
wave2 = wave2 + 1*(x_coarse<x_0)

wave4 = reflected_wave(x_coarse,t,w,N=4) + transmitted_wave(x_coarse,t,w,N=4)
wave4 = wave4 + 1*(x_coarse<x_0)

In [ ]:
plt.figure(figsize=(10,6))
leftpart = np.where(x_coarse<0)
rightpart = np.where(x_coarse>x_r)

plt.plot(xc,q[0,:],'-k',zorder=11);
plt.plot(x_coarse[leftpart],wave2[leftpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[leftpart],wave4[leftpart],'b',lw=3,alpha=0.5);
plt.plot(x_coarse[rightpart],wave2[rightpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[rightpart],wave4[rightpart],'b',lw=3,alpha=0.5);
ax = plt.gca()
ymax, ymin = ax.get_ylim()
xmid = np.linspace(0,1)
plt.fill_between(xmid,ymax,ymin,color='lightgray',alpha=1.,zorder=10)
plt.autoscale(enable=True, axis='y', tight=True)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend(['exact','Up to 2 reflections','Up to 4 reflections'],fontsize=15,loc='best');

# Figure 11(b)

In [ ]:
w=1.
claw, Z, c = setup(mx=40000,w=w,Z_l=Z_l,Z_r=Z_r,
                c_l=c_l,c_r=c_r,x_0=x_0,x_r=x_r,oscillate=oscillate)
xc = claw.solution.state.grid.x.centers

claw.run()

In [ ]:
i = -1
skip = 100
q = claw.frames[i].q
t = claw.frames[i].t
x_coarse = xc[::skip]
wave2 = reflected_wave(x_coarse,t,w,N=2) + transmitted_wave(x_coarse,t,w,N=2)

wave4 = reflected_wave(x_coarse,t,w,N=4) + transmitted_wave(x_coarse,t,w,N=4)

In [ ]:
plt.figure(figsize=(10,6))
leftpart = np.where(x_coarse<0)
rightpart = np.where(x_coarse>x_r)

plt.plot(xc,q[0,:],'-k',zorder=11);
plt.plot(x_coarse[leftpart],wave2[leftpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[leftpart],wave4[leftpart],'b',lw=3,alpha=0.5);
plt.plot(x_coarse[rightpart],wave2[rightpart],'r--',lw=3)#,alpha=0.5);
plt.plot(x_coarse[rightpart],wave4[rightpart],'b',lw=3,alpha=0.5);
ax = plt.gca()
ymax, ymin = ax.get_ylim()
xmid = np.linspace(0,1)
plt.fill_between(xmid,ymax,ymin,color='lightgray',alpha=1.,zorder=10)
plt.autoscale(enable=True, axis='y', tight=True)
plt.autoscale(enable=True, axis='x', tight=True)
plt.legend(['exact','Up to 2 reflections','Up to 4 reflections'],fontsize=15,loc='best');